### ML Engineering

На основе данных, полученных на предыдущем этапе работы (Data Engineering), необходимо построить модель для предсказания CTR (Отношение числа кликов к числу просмотров)

Исходные данные:

Файлы - train.parquet, test.parquet, validate.parquet
https://github.com/hadjdeh/Stepik_Academy_Big_Data_for_DS/tree/master/ML_Engineering

Струтура данных:

<b>ad_id</b>	integer	id рекламного объявления

<b>target_audience_count</b>	decimal	размер аудитории, на которую таргетируется 

<b>has_video</b>integer	1 если есть видео, иначе 0

<b>is_cpm</b>	integer	1 если тип объявления CPM, иначе 0

<b>is_cpc</b>	integer	1 если тип объявления CPC, иначе 0

<b>ad_cost</b>	double	стоимость объявления в рублях

<b>day_count</b>	integer	Число дней, которое показывалась реклама

<b>ctr	double</b>	Отношение числа кликов к числу просмотров

### Задание:
    
Необходимо реализовать две PySpark задачи:

1) <b>PySparkMLFit.py</b> - задача, которая должна тренировать модель на входящих данных, сохранять ее и производить оценку качества модели, используя RegressionEvaluator и выводя в консоль RMSE модели на основе тестового датасета.

Варианты запуска задачи:

<b>spark-submit PySparkMLFit.py train.parquet test.parquet</b>

где:
train.parquet - путь к датасету, который необходимо использовать для обучения

test.parquet - путь к датасету, который необходимо использовать для оценки полученной модели 

2) <b>PySparkMLPredict.py</b> - задача, которая должна загружать модель и строить предсказание над переданными ей данными.

Варианты запуска задачи:

<b>spark-submit PySparkMLPredict.py validate.parquet result</b>

где:

validate.parquet - путь к датасету, для которого необходимо сделать предсказание

result - путь, по которому будет сохранен результат предсказаний в формате CSV следующего вида [ads_id, prediction]

#### Импортируем необходимые библиотеки 

In [5]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression , DecisionTreeRegressor, RandomForestRegressor,GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
import random
from pyspark.ml.pipeline import PipelineModel

#### Создаем Spark-сессию

In [6]:
spark = SparkSession.builder.appName("PySparkML").getOrCreate()

In [7]:
spark

#### Определяем пути до train, test, validate датасетов и считываем данные

In [8]:
PATH_TO_TRAIN=r'C:\Users\hadjd\stepik-ds-course-master\Week5\SparkML\Project\train.parquet'
PATH_TO_TEST=r'C:\Users\hadjd\stepik-ds-course-master\Week5\SparkML\Project\test.parquet'
PATH_TO_VALIDATE=r'C:\Users\hadjd\stepik-ds-course-master\Week5\SparkML\Project\validate.parquet'

DF_train=spark.read.parquet(PATH_TO_TRAIN)
DF_test=spark.read.parquet(PATH_TO_TEST)

In [9]:
DF_train.printSchema()

root
 |-- ad_id: integer (nullable = true)
 |-- target_audience_count: double (nullable = true)
 |-- has_video: integer (nullable = true)
 |-- is_cpm: integer (nullable = true)
 |-- is_cpc: integer (nullable = true)
 |-- ad_cost: double (nullable = true)
 |-- day_count: integer (nullable = true)
 |-- ctr: double (nullable = true)



In [10]:
DF_test.printSchema()

root
 |-- ad_id: integer (nullable = true)
 |-- target_audience_count: double (nullable = true)
 |-- has_video: integer (nullable = true)
 |-- is_cpm: integer (nullable = true)
 |-- is_cpc: integer (nullable = true)
 |-- ad_cost: double (nullable = true)
 |-- day_count: integer (nullable = true)
 |-- ctr: double (nullable = true)



In [11]:
print(DF_train.count())
print(DF_test.count())

99931
50016


### 1. Реализация задачи PySparkMLFit

In [16]:
def estimator_pipeline(train_dataframe, test_dataframe):
    
    random.seed(0)
    
    #создаем вектор features из всех колонок, кроме целевой метки ctr
    vector=VectorAssembler(inputCols=train_dataframe.columns[:-1], outputCol='features')
    
    #estimator LR с входными параметрами из задани
    estimator_LR = LinearRegression(featuresCol='features',
                      labelCol='ctr',
                      maxIter=40,
                      regParam=0.4,
                      elasticNetParam=0.8)
    
    #другие эстиматоры с параметрами по умолчанию
    estimator_DT=DecisionTreeRegressor(featuresCol='features',labelCol='ctr')
    estimator_RF=RandomForestRegressor(featuresCol='features', labelCol='ctr')
    estimator_GB=GBTRegressor(featuresCol='features', labelCol='ctr')
    
    #evaluator
    RMSE_evaluator=RegressionEvaluator(predictionCol='prediction',labelCol='ctr',metricName='rmse')
    
    # модели и результаты будем записывать в списки
    models=[]
    RMSE_result=[]
    
    #обучаем все эстиматоры
    for est_r in [estimator_LR,estimator_DT,estimator_RF,estimator_GB]:
        
        pipeline=Pipeline(stages=[vector,est_r])
        
        model=pipeline.fit(train_dataframe)
            
        models.append(model)
        
        #делаем прогноз по тестовому датасету
        prediction=pipeline.fit(train_dataframe).transform(test_dataframe)
        
        #считаем метрику RMSE для тестового датасета
        RMSE=round(RMSE_evaluator.evaluate(prediction),4)
        
        #записываем результат в массив для отображения в консоли
        RMSE_result.append(RMSE)
        
        print('Model: {}, \tRMSE = {}'.format(est_r,RMSE))
            
    #сохранение моделей в соответствующих директориях
    for pair in zip(models,['LR_model','DT_model','RF_model','GB_model']):
        pair[0].save(pair[1])
    
    return models, RMSE_result

#### Обучаем модели

In [17]:
models, results = estimator_pipeline(DF_train,DF_test)

Model: LinearRegression_d8141f38c3ca, 	RMSE = 0.4097
Model: DecisionTreeRegressor_56ec07bc42b6, 	RMSE = 0.0899
Model: RandomForestRegressor_5c5d8e156fae, 	RMSE = 0.1255
Model: GBTRegressor_607ff3eae79c, 	RMSE = 0.0719


В результате, в локальной дирректории создались папки:
    
- /LR_model
- /DT_model
- /RF_model
- /GB_model

В каждую из которых записалась информация о соответстующей модели.

Реализация py файла - PySparkMLFit

https://github.com/hadjdeh/Stepik_Academy_Big_Data_for_DS/blob/master/ML_Engineering/PySparkMLFit.py



### 2. Реализация задачи PySparkMLPredict

In [19]:
def predict(input_file,output_file):
    for model_name in ['LR_model','DT_model','RF_model','GB_model']:
        model=PipelineModel.load(model_name)
        prediction=model.transform(input_file)
        prediction[['ad_id', 'prediction']].coalesce(1).write.option('header', 'true').csv(output_file+'/'+model_name)

#### Считываем валидационный датасет

In [20]:
DF_validate=spark.read.parquet(PATH_TO_VALIDATE)

#### Делаем prediction

In [21]:
predict(DF_validate,'result')

В результате в локальной директории создалась папка:

- /result

В которую был записан csv файл с предсказаниями для каждой модели.

Реализация PySparkMLPredict.py файла 

https://github.com/hadjdeh/Stepik_Academy_Big_Data_for_DS/blob/master/ML_Engineering/PySparkMLPredict.py

